### Putting Files in the Cloud!

In [ ]:
import boto3
import pandas as pd

Creating user client

In [ ]:
# Generate the boto3 client for interacting with S3
s3 = boto3.client(# AWS Services.
                  's3',
                  region_name='us-east-1', 
                  # Set up AWS credentials KeyID 'Username' Key 'Password'
                  aws_access_key_id=AWS_KEY_ID, 
                  aws_secret_access_key=AWS_SECRET)
                  
# List the buckets
buckets = s3.list_buckets()

# Print the buckets
print(buckets)

Creating multiple Clients

In [ ]:
# Generate the boto3 client for interacting with S3 and SNS
s3 = boto3.client('s3',
                  region_name='us-east-1', 
                  aws_access_key_id=AWS_KEY_ID, 
                  aws_secret_access_key=AWS_SECRET)

sns = boto3.client('sns',
                   region_name='us-east-1', 
                   aws_access_key_id=AWS_KEY_ID, 
                   aws_secret_access_key=AWS_SECRET)

# List S3 buckets and SNS topics
buckets = s3.list_buckets()
topics = sns.list_topics()

# Print out the list of SNS topics
print(topics)

Creating buckets

In [ ]:
import boto3

# Create boto3 client to S3
s3 = boto3.client('s3',
                  region_name='us-east-1', 
                  aws_access_key_id=AWS_KEY_ID, 
                  aws_secret_access_key=AWS_SECRET)

# Create the buckets
response_staging = s3.create_bucket(Bucket='gim-staging')
response_processed = s3.create_bucket(Bucket='gim-processed')
response_test = s3.create_bucket(Bucket='gim-test')

# Print out the response
print(response_staging)

List all buckets

In [ ]:
# Get the list_buckets response
response = s3.list_buckets()

# Iterate over Buckets from .list_buckets() response
for bucket in response['Buckets']:
  
  	# Print the Name for each bucket
    print(bucket['Name'])

Delete bucket

In [ ]:
# Delete the gim-test bucket
s3.delete_bucket(Bucket='gim-test')

Insert Object into bucket (Upload)

In [ ]:
# Upload final_report.csv to gid-staging
s3.upload_file(Bucket='gid-staging',
              # Set filename and key
               Filename='final_report.csv', 
               Key='2019/final_report_01_01.csv')

List all objects inside bucket

In [ ]:
response = s3.list_objects(
    Bucket='gid-staging',
    Prefix='2018/'
)

Get metadata from object

In [ ]:
# Get object metadata and print it
response = s3.head_object(Bucket='gid-staging', 
                          Key='2019/final_report_01_01.csv')

# Print the size of the uploaded object
print(response['ContentLength'])

Download File 

In [ ]:
s3.download_file(
    Filename='final_report_01_02.csv',
    Bucket='gid-staging',
    Key='2019/final_report_01_02.csv'
)

Delete File

In [ ]:
s3.delete_object(
    Bucket='gid-staging',
    Key='2019/final_report_01_01.csv'
)

## Sharing Files Securely

Upload file with puclic-read ACL

In [ ]:
# Upload the final_report.csv to gid-staging bucket
s3.upload_file(
  # Complete the filename
  Filename='./final_report.csv', 
  # Set the key and bucket
  Key='2019/final_report_2019_02_20.csv', 
  Bucket='gid-staging',
  # During upload, set ACL to public-read
  ExtraArgs = {'ACL': 'public-read'})

Configure ACL of existing object

In [ ]:
# Give each object ACL of public-read
s3.put_object_acl(Bucket='gid-staging', 
                  Key='2019/final_report_2019_02_20.csv', 
                  ACL='public-read')
                 #ACL='private'  

Public Object URL

In [ ]:
# Print the Public Object URL for each object
url = "https://{}.s3.amazonaws.com/{}".format('bucket_name', 'key_name') 

print(url)

# Reading csv inside url
df = pd.read_csv(url)

#### Accessing Private Files

Can Download and Then used from disk

In [ ]:
# Download file to computer
s3.download_file(
    Filename='potholes_local.csv',
    Bucket='gig-staging',
    Key='2019/potholes_local.csv'
)

# Read it from disk
pd.read_csv('./potholes_local.csv')

Accessing Private Files

In [ ]:
# Access private object
obj = s3.get_object(Bucket='gid-requests', Key='KeyName')

# Returns Metadata + Obj content Inside BODY
print(obj)

# Read csv with obj
df = pd.read_csv(obj['Body'])

Generate Pre-asign URL

In [ ]:
# Generate presigned_url for the uploaded object
share_url = s3.generate_presigned_url(
  # Specify allowable operations
  ClientMethod='get_object',
  # Set the expiration time
  ExpiresIn=3600,
  # Set bucket and shareable object's name
  Params={'Bucket': 'gid-staging','Key': 'final_report.csv'}
)

# Print out the presigned URL
print(share_url)

# Example
# https://gid-staging.s3.amazonaws.com/final_report.csv?AWSAccessKeyId=IAmAFakeKey&Signature=dshH9q7LgMLHlbd5vWXdN4X4OSc%3D&Expires=1660178422

# Read with pandas
df = pd.read_csv(share_url)


#### HTML to S3

Creating Html Object with pandas

In [ ]:
# Convert Dataframe to html
services_df.to_html('./services_no_border.html',
           # Keep specific columns only
           columns=['service_name', 'link'],
           # Set border
           border=0,
           render_links=True)

Upload Html object to s3

In [ ]:
# Upload the lines.html file to S3
s3.upload_file(Filename='./lines.html', 
               # Set the bucket name
               Bucket='datacamp-public',
               Key='index.html',
               # Configure uploaded file
               ExtraArgs = {
                 # Set proper content type
                 'ContentType':'text/html',
                 # Set proper ACL
                 'ACL': 'public-read'})

# Print the S3 Public Object URL for the new file.
print("http://{}.s3.amazonaws.com/{}".format('datacamp-public', 'index.html'))

Upload Images to S3

In [ ]:
# Upload the lines.html file to S3
s3.upload_file(Filename='./plot_image.png', 
               Bucket='datacamp-public',
               Key='plot_image.png',
               ExtraArgs = {
                 'ContentType':'image/png',
                 'ACL': 'public-read'})

# More FileTypes IANA Media Types.
# JSON : application/json
# PDF : application/pdf
# CSV : text/csv


Example of report creation with Html and s3

In [ ]:
df_list = [] 

# List all files inside bucket prefix 2018
request_files = response = s3.list_objects(
    Bucket='gid-staging',
    Prefix='2018/'
)

# Load each object from s3
for file in request_files:
    s3_day_reqs = s3.get_object(Bucket='gid-requests', 
                                Key=file['Key'])

    # Read the DataFrame into pandas, append it to the list
    day_reqs = pd.read_csv(s3_day_reqs['Body'])
    df_list.append(day_reqs)

# Concatenate all the DataFrames in the list
all_reqs = pd.concat(df_list)

# Preview the DataFrame
all_reqs.head()

In [ ]:
# Write agg_df to a CSV and HTML file with no border
agg_df.to_csv('./feb_final_report.csv')
agg_df.to_html('./feb_final_report.html', border=0)

# Upload the generated CSV to the gid-reports bucket
s3.upload_file(Filename='./feb_final_report.csv', 
	Key='2019/feb/final_report.html',
    Bucket='gid-reports',
    ExtraArgs = {'ACL': 'public-read'})

# Upload the generated HTML to the gid-reports bucket
s3.upload_file(Filename='./feb_final_report.html', 
	Key='2019/feb/final_report.html',
    Bucket='gid-reports',
    ExtraArgs = {'ContentType': 'text/html', 
                 'ACL': 'public-read'})

In [ ]:
# List the gid-reports bucket objects starting with 2019/
objects_list = s3.list_objects(Bucket='gid-reports', Prefix='2019/')

# Convert the response contents to DataFrame
objects_df = pd.DataFrame(objects_list['Contents'])

# Create a column "Link" that contains Public Object URL
base_url = "http://gid-reports.s3.amazonaws.com/"
objects_df['Link'] = base_url + objects_df['Key']

# Preview the resulting DataFrame
objects_df.head()

In [ ]:
# Write objects_df to an HTML file
objects_df.to_html('report_listing.html',
    # Set clickable links
    render_links=True,
	# Isolate the columns
    columns=['Link', 'LastModified', 'Size'])

# Overwrite index.html key by uploading the new file
s3.upload_file(
  Filename='./report_listing.html',
  Key='index.html', 
  Bucket='gid-reports',
  ExtraArgs = {
    'ContentType': 'text/html', 
    'ACL': 'public-read'
  })

## Reporting and Notifying!

Creating SNS topic

In [ ]:
# Create boto3 client to SNS
sns = boto3.client('sns',
                   region_name='us-east-1',
                   aws_access_key_id=AWS_KEY_ID,
                   aws_secret_access_key=AWS_SECRET)

# Create a Topic. Returns API response
response = sns.create_topic(Name='city_alerts')

# Print topic 'id'
print(response['TopicArn'])

List Topics

In [ ]:
response = sns.list_topics()

Delete Topic

In [ ]:
sns.delete_topic(TopicArn='arn:aws:sns:us-east-1:43424235235234:city_alerts')

Subscribe to a Topic

In [ ]:
# Subscribe some cellphone to topic
response = sns.subscribe(TopicArn='arn:aws:sns:us-east-1:43424235235234:city_alerts',
                         Protocol='SMS',
                         Endpoint='+13125551123')

# Subscribe some email to topic
response = sns.subscribe(TopicArn='arn:aws:sns:us-east-1:43424235235234:city_alerts',
                         Protocol='email',
                         Endpoint='something@gmail.com')

List subscriptions by Topic

In [ ]:
sns.list_subscriptions_by_topic(TopicArn='arn:aws:sns:us-east-1:43424235235234:city_alerts')

List All subscriptions

In [ ]:
sns.list_subscriptions()['Subscriptions']

UnSubscribe to a topic

In [ ]:
sns.unsubscribe(SubscriptionArn='arn:aws:sns:us-east-1:43424235235234:city_alerts:9fea12d213ad14ae')

Publish a menssage to Topic

In [ ]:
sns.publish(TopicArn='arn:aws:sns:us-east-1:43424235235234:city_alerts',
            Message='Body text of SMS or Email',
            Subject='Subject Line for Email')

# Custom messages
n_reports = 134
sns.publish(TopicArn='arn:aws:sns:us-east-1:43424235235234:city_alerts',
            Message='There are {} reports outstanding'.format(n_reports),
            Subject='Subject Line for Email')

Sending a simple SMS message without Topics or Subscribers (This is not good in the long run)

In [ ]:
sns.publish(PhoneNumber='+121332312514',
            Message='Body of text of SMS')

Case Building notification System

In [ ]:
# Topic Setup
sns = boto3.client('sns',
                   region_name='us-east-1',
                   aws_access_key_id=AWS_KEY_ID,
                   aws_secret_access_key=AWS_SECRET)

# Create Topics
trash_arn = sns.create_topic(Name='trash_notifications')['TopicArn']
streets_arn = sns.create_topic(Name='street_notification')['TopicArn']

# Subscribe Users to Topics
contacts = pd.read_csv('http://gig-staging.contacts.csv') # File stored in S3

# Create Function to help with subscribing
def subscribe_user(user_row):
    if user_row['Department'] == 'trash':
        # SMS 
        sns.subscribe(TopicArn=trash_arn,
                      Protocol='sms',
                      Endpoint=str(user_row['Phone']))
        # Email
        sns.subscribe(TopicArn=trash_arn,
                      Protocol='email',
                      Endpoint=user_row['Email'])

    else:
         # SMS 
        sns.subscribe(TopicArn=streets_arn,
                      Protocol='sms',
                      Endpoint=str(user_row['Phone']))
        # Email
        sns.subscribe(TopicArn=streets_arn,
                      Protocol='email',
                      Endpoint=user_row['Email'])

# Applying function to users dataframe
contacts.apply(subscribe_user, axis=1)

# Getting aggregate dataframe 
df = pd.read_csv('http://gig-reports.2019/feb/final_reports.csv')
df.set_index('service_name', inplace=True)

trash_violations_count = df.at['Illegal Dumping','count']
streets_violations_count = df.at['Pothole','count']

# Send Alerts
if trash_violations_count > 100:
    message = 'Trash violations count is now {}'.format(trash_violations_count)
    sns.publish(TopicArn=trash_arn,
                Message=message,
                Subject='Trash Alert')

if streets_violations_count > 100:
    message = 'Street violations count is now {}'.format(streets_violations_count)
    sns.publish(TopicArn=streets_arn,
                Message=message,
                Subject='Street Alert')
